<a href="https://colab.research.google.com/github/bhaveshsingh0206/nlp/blob/main/Preprocess_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bs4 # in case you don't have it installed
!pip install contractions

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
import contractions
import collections
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/NLP\ Dataset/

## Yahoo Dataset

In [ ]:
!pip install yahooquery

In [ ]:
# !pip install yahooquery

from datetime import datetime, timedelta
import requests
from yahooquery import Ticker

def date_range(current_date, num_days=10):
    """Helper function to get x number of days before and after the current date

    Args:
        current_date (str): Current date in YYYY-MM-DD format
        num_days (int): Number of days by which the `current_date` is incremented and decremented

    Returns:
        list: List of dates in the sepcified interval
    """
    num_days = num_days + 1 # Adding 1 because the current date is included in the interval
    base = datetime.strptime(current_date, '%Y-%m-%d')

    
    # Previous `x` days --> 06, 05, 04 (base is 06)
    date_list_prev = [(base - timedelta(days=x)).strftime('%Y-%m-%d') for x in range(num_days)]
    # Next `x` days     --> 06, 07, 08 (base is 06)
    date_list_next = [(base + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(num_days)]
    
    # [06, 05, 04][::-1] + [06, 07, 08][1:]
    # return date_list_prev[::-1] + date_list_next[1:]
    return date_list_prev[::-1]

def get_close_prices(ticker, current_date, num_days=10):
    """Return a list of close prices for every day in the given range of startdate and enddate

    Args:
        ticker (str): Company ticker
        current_date (str): Date (YYYY-MM-DD) for which we need the previous and next 10 close prices
        num_days (str): Number of days offset. Defaults to 10

    Returns:
        list: List of close prices
    """
    tuples = []
    close_prices = []
    first_open = 0

    # Get list of dates for which close price is to be calculated
    dates = date_range(current_date=current_date, num_days=num_days)
    start_date = dates[0]
    end_date = dates[-1]

    # Get close prices
    ticker = ticker.lower()
    ticker = Ticker(ticker, asynchronous=True)
    df = ticker.history(start=start_date, end=end_date)

    # Group data -> [(date, price), (date, price), ...]
    for index, row in df.iterrows():
        # To handle the case where the first price is zero
        # Initialize with open price of the next day
        if first_open == 0:
            first_open = row['open']
        date = index[1].strftime('%Y-%m-%d')
        tuples.append((date, row['close']))
    
    curr_price = first_open
    
    # `dates` is a super set of `close_prices`
    # For every date if there is no close price we take the previus price
    j = 0
    for i in range(len(dates)):
        if j < len(tuples) and tuples[j][0] == dates[i]:
            curr_price = tuples[j][1]
            close_prices.append(curr_price)
            j = j + 1
        else:
            close_prices.append(curr_price)

    return close_prices

def yahoo_data(current_date):
    # Date for which we need the previous and next 10 close prices
    # current_date = '1999-06-06'
    # Number of days by which the `current_date` is incremented and decremented
    num_days = 10
    
    prices = get_close_prices(ticker='^gspc', current_date=current_date, num_days=num_days)
    # print('Length: {}\n\nPrices:\n{}'.format(len(prices), prices))
    return prices


## Create Reuters and Bloomberg CSV

In [ ]:
newdf = pd.DataFrame(columns=['ts', 'title', 'src'])
for filename in tqdm(os.listdir("./reuters 2/")):
  df = pd.read_csv("reuters 2/" + filename, sep='\t', usecols=[0,1])
  df['src'] = 'reuters'
  newdf = newdf.append(df, ignore_index=True)

In [ ]:
newdf

In [ ]:
for idx, v in enumerate(newdf['ts']):
  date = v.split(' ')[0]
  y = date[:4]
  m = date[4:6]
  d = date[6:]
  newdf.at[idx, 'ts'] = y + "-" + m + "-" + d

In [ ]:
newdf

In [ ]:
newdf.to_csv('./reuters.csv')

In [ ]:
# !unzip ./20061020_20131126_bloomberg_news\ 2.zip

In [ ]:
res = []

for folder in tqdm(sorted(os.listdir("./20061020_20131126_bloomberg_news 2/"), reverse=True)):
  for filename in os.listdir("./20061020_20131126_bloomberg_news 2/"+folder):
    file1 = open('./20061020_20131126_bloomberg_news 2/'+folder+'/'+filename, 'r')
    lines = file1.readlines()
    row = []
    for idx, line in enumerate(lines):
      if idx == 0:
        row.append(line[3:-1])
      if idx == 2:
        row.append(line.split('T')[0][3:])
        break
    row = row[::-1]
    res.append(row)

In [ ]:
len(res)

In [ ]:
bloomberg_df = pd.DataFrame(res, columns = ['ts', 'title', 'src'])

In [ ]:
bloomberg_df

In [ ]:
bloomberg_df.to_csv('./bloomberg.csv')

## Merge Data

In [ ]:
bloomberg_df = pd.read_csv('./bloomberg.csv', index_col=0)
bloomberg_df

In [ ]:
reuters_df = pd.read_csv('./reuters.csv', index_col=0)
reuters_df

In [ ]:
merged_df = pd.DataFrame(columns=['ts', 'title', 'src'])
merged_df = merged_df.append(bloomberg_df, ignore_index=True)
merged_df = merged_df.append(reuters_df, ignore_index=True)
merged_df

In [ ]:
merged_df.shape

In [ ]:
merged_df.to_csv('../merged.csv')

## Cleaning Data

In [ ]:
bloomberg_df = pd.read_csv('./bloomberg.csv', index_col=0)
bloomberg_df

In [ ]:
reuters_df = pd.read_csv('./reuters.csv', index_col=0)
reuters_df

In [ ]:
merged_df = pd.read_csv('./merged.csv', index_col=0)
merged_df

In [ ]:
def calculateAverage(df, reviewBody, cleanBody):
  originalUncleanedAverage=df[reviewBody].apply(lambda x: len(x)).mean()
  originalCleanedAverage=df[cleanBody].apply(lambda x: len(x)).mean()
  print(f'Original uncleaned reviews had an average length of ${originalUncleanedAverage}')
  print(f'New cleaned reviews had an average length of ${originalCleanedAverage}')

In [ ]:
punctuations = '''!()-[]{};:"\,<>./?@#%^&*_~'''
def cleanText(x):
  x = x.lower().strip()
  soup = BeautifulSoup(x)
  x = soup.get_text()
  x = re.sub(r'https?://\S+', '', x)
  x=re.sub("\s\s+", " ", x.strip())
  # x=re.sub("[^a-zA-Z\s]+", " ", x)

  no_punct = ""
  for char in x:
    if char not in punctuations:
        no_punct = no_punct + char
    else:
        no_punct += " "
  x = no_punct

  x=re.sub("\s\s+", " ", x.strip())
  if (x==' ' or len(x)==0): 
    return np.nan
  return x

In [ ]:
reuters_df['clean_title'] = reuters_df['title'].apply(cleanText)

In [ ]:
reuters_df.dropna(inplace=True)
reuters_df.reset_index(inplace=True, drop=True)

In [ ]:
reuters_df

In [ ]:
calculateAverage(reuters_df, 'title', 'clean_title')

In [ ]:
bloomberg_df.dropna(inplace=True)

In [ ]:
bloomberg_df['clean_title'] = bloomberg_df['title'].apply(cleanText)
bloomberg_df.dropna(inplace=True)
bloomberg_df.reset_index(inplace=True)
bloomberg_df

In [ ]:
calculateAverage(bloomberg_df, 'title', 'clean_title')

In [ ]:
bloomberg_df = bloomberg_df.drop('index', axis=1)

In [ ]:
bloomberg_df

In [ ]:
reuters_df.to_csv('./reuters_clean.csv')
bloomberg_df.to_csv('./bloomberg_clean.csv')

In [ ]:
merged_df = pd.DataFrame(columns=['ts', 'title', 'src', 'clean_title'])
merged_df = merged_df.append(bloomberg_df, ignore_index=True)
merged_df = merged_df.append(reuters_df, ignore_index=True)
merged_df

In [ ]:
merged_df.to_csv('./merged_clean.csv')

## Creating final data with Yahoo values

In [ ]:
merged_df = pd.read_csv('./merged_clean.csv', index_col=0)
merged_df

In [ ]:
df2 = merged_df.sort_values('ts', ascending=False, ignore_index=True)

In [ ]:
df2

In [ ]:
df2=df2.drop(df2.index[[0,1,2]])
df2.reset_index(inplace=True)

In [ ]:
df2 = df2.drop('index', axis=1)
df2

In [ ]:
df2.to_csv('./merged_clean_sorted.csv')

In [ ]:
df2 = pd.read_csv('./merged_clean_sorted.csv', index_col=0)
df2

In [ ]:
df2['ts'].value_counts().sort_values(ascending=True) 

In [ ]:
len(df2['ts'].value_counts())

In [ ]:
m = collections.defaultdict(list)
cnt = collections.defaultdict(lambda: 0)

for index, row in tqdm(df2.iterrows()):
    if cnt[row['ts']]>400:
      continue
    else:
      if row['ts'] in m:
        for idx, i in enumerate(m[row['ts']]):
          df2.loc[index,'p'+str(idx)] = i
      else:
        yd = yahoo_data(row['ts'])
        for idx, i in enumerate(yd):
          df2.loc[index,'p'+str(idx)] = i
          m[row['ts']] = yd
      cnt[row['ts']] += 1

In [ ]:
df2

In [ ]:
m = collections.defaultdict(list)
cnt = collections.defaultdict(lambda: 0)

for index, row in tqdm(df2.iterrows()):
    if cnt[row['ts']]>400:
      continue
    else:
      if row['ts'] in m:
        for idx, i in enumerate(m[row['ts']]):
          df2.loc[index,'p'+str(idx)] = i
      else:
        yd = yahoo_data(row['ts'])
        for idx, i in enumerate(yd):
          df2.loc[index,'p'+str(idx)] = i
          m[row['ts']] = yd
      cnt[row['ts']] += 1

In [ ]:
df2 = df2.dropna()
df2.reset_index(inplace=True)

In [ ]:
df2 = df2.drop('index', axis=1)
df2

In [ ]:
df2.to_csv('./final_data.csv')